In [22]:
# install mctest package
install.packages('mctest', repos='http://cran.us.r-project.org')

In [ ]:
library(foreign) # library for importing sav file (SPSS) into R.
library(mctest) # package for running multicolinearity tests

In [15]:
dataset = read.spss("./data/cars.sav", to.data.frame=TRUE)

In [16]:
str(dataset)

'data.frame':	81 obs. of  8 variables:
 $ Manufacturer: Factor w/ 32 levels "Acura                                     ",..: 1 1 2 2 3 4 4 4 4 5 ...
 $ Model       : Factor w/ 81 levels "100                                       ",..: 43 48 1 8 5 20 50 62 63 30 ...
 $ Type        : Factor w/ 5 levels "Compact              ",..: 4 3 3 1 3 3 2 3 2 2 ...
 $ Price       : num  15.9 33.9 37.7 29.1 30 15.7 20.8 26.3 23.7 34.7 ...
 $ Citympg     : num  25 18 19 20 22 22 19 19 16 16 ...
 $ EngineSize  : num  1.8 3.2 2.8 2.8 3.5 2.2 3.8 3.8 5.7 4.9 ...
 $ Horsepower  : num  140 200 172 172 208 110 170 170 180 200 ...
 $ Weight      : num  2705 3560 3405 3375 3640 ...
 - attr(*, "codepage")= int 65001


In [17]:
# Add extra features
dataset$Horsepower2 <- dataset$Horsepower^2
dataset$Citympg2 <- dataset$Citympg^2

In [24]:
str(dataset)

'data.frame':	81 obs. of  10 variables:
 $ Manufacturer: Factor w/ 32 levels "Acura                                     ",..: 1 1 2 2 3 4 4 4 4 5 ...
 $ Model       : Factor w/ 81 levels "100                                       ",..: 43 48 1 8 5 20 50 62 63 30 ...
 $ Type        : Factor w/ 5 levels "Compact              ",..: 4 3 3 1 3 3 2 3 2 2 ...
 $ Price       : num  15.9 33.9 37.7 29.1 30 15.7 20.8 26.3 23.7 34.7 ...
 $ Citympg     : num  25 18 19 20 22 22 19 19 16 16 ...
 $ EngineSize  : num  1.8 3.2 2.8 2.8 3.5 2.2 3.8 3.8 5.7 4.9 ...
 $ Horsepower  : num  140 200 172 172 208 110 170 170 180 200 ...
 $ Weight      : num  2705 3560 3405 3375 3640 ...
 $ Horsepower2 : num  19600 40000 29584 29584 43264 ...
 $ Citympg2    : num  625 324 361 400 484 484 361 361 256 256 ...
 - attr(*, "codepage")= int 65001


In [19]:
fitted_model <- lm(Price ~ Citympg + Citympg2 + EngineSize + Horsepower + Horsepower2+  Weight, dataset)

In [20]:
anova(fitted_model)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
Citympg,1,2743.500653,2743.500653,121.5739256,2.810266e-17
Citympg2,1,1065.357242,1065.357242,47.2096341,1.709079e-09
EngineSize,1,10.704627,10.704627,0.4743587,4.931425e-01
Horsepower,1,640.750010,640.750010,28.3938311,1.028033e-06
Horsepower2,1,3.394763,3.394763,0.1504336,6.992348e-01
Weight,1,52.268851,52.268851,2.3162121,1.322934e-01
Residuals,74,1669.922620,22.566522,NA,NA


In [ ]:
# TODO: MCT tests, see what variables are colinear

In [25]:
eigprop(fitted_model) # all of the variables and data except the independent

ERROR: Error in eigprop(fitted_model): X matrix must contain more than one variable
